# Imports and NLP


In [ ]:
# !python -m spacy download en_core_web_lg

In [ ]:
import pandas as pd 
import numpy as np
import pickle
import spacy
from sklearn.neighbors import NearestNeighbors
import time
import re

In [ ]:
%load_ext google.colab.data_table

In [ ]:
import spacy.cli
spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


# Examine Data

In [ ]:
# @title Retrieve Data

url = "https://raw.githubusercontent.com/vjmiyagi/DS_BuildWeek_Dec_2020/main/csv/data.csv"
df_main = pd.read_csv(url)
df_artist = pd.read_csv("https://raw.githubusercontent.com/vjmiyagi/DS_BuildWeek_Dec_2020/main/csv/data_by_artist.csv")
df_genre = pd.read_csv("https://raw.githubusercontent.com/vjmiyagi/DS_BuildWeek_Dec_2020/main/csv/data_by_genres.csv")
df_year = pd.read_csv("https://raw.githubusercontent.com/vjmiyagi/DS_BuildWeek_Dec_2020/main/csv/data_by_year.csv")
df_w = pd.read_csv("https://raw.githubusercontent.com/vjmiyagi/DS_BuildWeek_Dec_2020/main/csv/data_w_genres.csv")

new_row = {"valence": 0.404, "year": 1962, "acousticness": 0.707, 
            "artists": "['Kyu Sakamoto']", "danceability": 0.415, 
            "duration_ms": 189520, "energy": 0.359, "explicit": 1, 
            "id": "6meIeOX3DHdaCnaNw67abE", "instrumentalness": 0.0000653,
            "key": 7, "liveness": 0.319, "loudness": -11.094, "mode": 1, 
            "name": "上を向いて歩こう Ue Wo Muite Arukou Sukiyaki", "popularity": 50,
            "release_date": "2010-01-01", "speechiness": 0.0334, "tempo": 146.029}

df_main = df_main.append(new_row, ignore_index=True)


In [ ]:
#@title Heads or Tails  { run: "auto" }
show = "Heads" #@param ["Heads", "Tails"]
from __future__ import print_function

from google.colab import widgets
from google.colab import output


a = df_main.head()
b = df_main.tail()
c = df_artist.head()
d = df_artist.tail()
e = df_genre.head()
f = df_genre.tail()
g = df_year.head()
h = df_year.tail()
i = df_year.head()
j = df_year.tail()

heads = [a, c, e, f, i]
tails = [b, d, f, h, j]

def TB():
    tb = widgets.TabBar(["Data", "Artist", "Genre", "Year", "Genre with Artist"])
    l_tab = show
    for x in range(5):
        with tb.output_to(x):
            if l_tab == "Heads":
                print(heads[x])
            elif l_tab == "Tails":
                print(tails[x])

dash = '=' * 81
print(dash)
t7 = "Shapes"
print(f'{t7:^81}')
print()
s1, s2, s3, s4, s5 = ("Main", "Artist", "Genre",
                              "Year", "Genre with Artist")
print(f'{s1:^16}{s2:^16}{s3:^16}{s4:^16}{s5:^16}')
t1, t2, t3, t4, t5 = (str(df_main.shape), str(df_artist.shape), 
                          str(df_genre.shape), str(df_year.shape),
                          str(df_w.shape))

print(f'{t1:^16}{t2:^16}{t3:^16}{t4:^16}{t5:^16}')
print()
print(dash)





TB()

                                     Shapes                                      

      Main           Artist          Genre            Year      Genre with Artist
  (170654, 19)    (28680, 15)      (2973, 14)      (100, 14)      (28680, 16)   



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

   mode  year  acousticness  ...   valence  popularity  key
0     1  1921      0.886896  ...  0.379327    0.653333    2
1     1  1922      0.938592  ...  0.535549    0.140845   10
2     1  1923      0.957247  ...  0.625492    5.389189    0
3     1  1924      0.940200  ...  0.663725    0.661017   10
4     1  1925      0.962607  ...  0.621929    2.604317    5

[5 rows x 14 columns]


<IPython.core.display.Javascript object>

In [ ]:
#@title Explore Data  { run: "auto" }
dataset = "Data" #@param ["Data", "Artist", "Genre", "Year", "Genre with Artist"]
if dataset == "Data":
    df = df_main
    title = "Data"
elif dataset == "Artist":
    df = df_artist
    title = "Data by Artist"
elif dataset == "Genre":
    df = df_genre
    title = "Data by Genre"
elif dataset == "Year":
    df = df_year
    title = "Data by Year"
else:
    df = df_w
    title = "Genre with Artist"
# pd.set_option("display.precision", 2)
dash = "="*50

print(f"{title:^50}")
print(dash)
df.info()

                       Data                       
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170654 entries, 0 to 170653
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   valence           170654 non-null  float64
 1   year              170654 non-null  int64  
 2   acousticness      170654 non-null  float64
 3   artists           170654 non-null  object 
 4   danceability      170654 non-null  float64
 5   duration_ms       170654 non-null  int64  
 6   energy            170654 non-null  float64
 7   explicit          170654 non-null  int64  
 8   id                170654 non-null  object 
 9   instrumentalness  170654 non-null  float64
 10  key               170654 non-null  int64  
 11  liveness          170654 non-null  float64
 12  loudness          170654 non-null  float64
 13  mode              170654 non-null  int64  
 14  name              170654 non-null  object 
 15  popularity       

In [ ]:
#@title Heads or Tails ... a closer look  { run: "auto" }
show = "Tails" #@param ["Heads", "Tails"]
from __future__ import print_function

from google.colab import widgets
from google.colab import output

z1 = len(df_main) - 1
z2 = len(df_artist) - 1
z3 = len(df_genre) - 1
z4 = len(df_year) - 1
z5 = len(df_w) - 1
a = df_main.loc[0]
b = df_main.loc[z1]
c = df_artist.loc[0]
d = df_artist.loc[z2]
e = df_genre.loc[0]
f = df_genre.loc[z3]
g = df_year.loc[0]
h = df_year.loc[z4]
i = df_w.loc[0]
j = df_w.loc[z5]

heads = [a, c, e, f, i]
tails = [b, d, f, h, j]

def TB():
    tb = widgets.TabBar(["Data", "Artist", "Genre", "Year", "Genre with Artist"])
    l_tab = show
    for x in range(5):
        with tb.output_to(x):
            if l_tab == "Heads":
                print(heads[x])
            elif l_tab == "Tails":
                print(tails[x])

dash = '=' * 81
print(dash)
t7 = "Shapes"
print(f'{t7:^81}')
print()
s1, s2, s3, s4, s5 = ("Main", "Artist", "Genre",
                              "Year", "Genre with Artist")
print(f'{s1:^16}{s2:^16}{s3:^16}{s4:^16}{s5:^16}')
t1, t2, t3, t4, t5 = (str(df_main.shape), str(df_artist.shape), 
                          str(df_genre.shape), str(df_year.shape),
                          str(df_w.shape))

print(f'{t1:^16}{t2:^16}{t3:^16}{t4:^16}{t5:^16}')
print()
print(dash)





TB()

                                     Shapes                                      

      Main           Artist          Genre            Year      Genre with Artist
  (170654, 19)    (28680, 15)      (2973, 14)      (100, 14)      (28680, 16)   



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

genres              ['classic korean pop']
artists                                조정현
acousticness                         0.568
danceability                         0.447
duration_ms                         237688
energy                               0.215
instrumentalness                  1.16e-06
liveness                            0.0649
loudness                           -16.478
speechiness                         0.0272
tempo                               71.979
valence                              0.177
popularity                              31
key                                     10
mode                                     1
count                                    2
Name: 28679, dtype: object


<IPython.core.display.Javascript object>

In [ ]:
#@title Heads or Tails ... a closer look  { run: "auto" }
show = "Tails" #@param ["Heads", "Tails"]
from __future__ import print_function

from google.colab import widgets
from google.colab import output

z1 = len(df_main) - 1
z2 = len(df_artist) - 1
z3 = len(df_genre) - 1
z4 = len(df_year) - 1
z5 = len(df_w) - 1
a = df_main.loc[0]
b = df_main.loc[z1]
c = df_artist.loc[0]
d = df_artist.loc[z2]
e = df_genre.loc[0]
f = df_genre.loc[z3]
g = df_year.loc[0]
h = df_year.loc[z4]
i = df_w.loc[0]
j = df_w.loc[z5]

heads = [a, c, e, f, i]
tails = [b, d, f, h, j]

def TB():
    tb = widgets.TabBar(["Data", "Artist", "Genre", "Year", "Genre with Artist"])
    l_tab = show
    for x in range(5):
        with tb.output_to(x):
            if l_tab == "Heads":
                print(heads[x])
            elif l_tab == "Tails":
                print(tails[x])

dash = '=' * 81
print(dash)
t7 = "Shapes"
print(f'{t7:^81}')
print()
s1, s2, s3, s4, s5 = ("Main", "Artist", "Genre",
                              "Year", "Genre with Artist")
print(f'{s1:^16}{s2:^16}{s3:^16}{s4:^16}{s5:^16}')
t1, t2, t3, t4, t5 = (str(df_main.shape), str(df_artist.shape), 
                          str(df_genre.shape), str(df_year.shape),
                          str(df_w.shape))

print(f'{t1:^16}{t2:^16}{t3:^16}{t4:^16}{t5:^16}')
print()
print(dash)





TB()

                                     Shapes                                      

      Main           Artist          Genre            Year      Genre with Artist
  (170654, 19)    (28680, 15)      (2973, 14)      (100, 14)      (28680, 16)   



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

genres              ['classic korean pop']
artists                                조정현
acousticness                         0.568
danceability                         0.447
duration_ms                         237688
energy                               0.215
instrumentalness                  1.16e-06
liveness                            0.0649
loudness                           -16.478
speechiness                         0.0272
tempo                               71.979
valence                              0.177
popularity                              31
key                                     10
mode                                     1
count                                    2
Name: 28679, dtype: object


<IPython.core.display.Javascript object>

In [ ]:
df

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.98200,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.73200,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.96100,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.96700,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.95700,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170649,0.7340,2020,0.20600,['Ashnikko'],0.717,150654,0.753,0,0OStKKAuXlxA0fMH54Qs6E,0.000000,7,0.101,-6.020,1,Halloweenie III: Seven Days,68,2020-10-23,0.0605,137.936
170650,0.6370,2020,0.10100,['MAMAMOO'],0.634,211280,0.858,0,4BZXVFYCb76Q0Klojq4piV,0.000009,4,0.258,-2.226,0,AYA,76,2020-11-03,0.0809,91.688
170651,0.1950,2020,0.00998,['Eminem'],0.671,337147,0.623,1,5SiZJoLXp3WOl3J4C8IK0d,0.000008,2,0.643,-7.161,1,Darkness,70,2020-01-17,0.3080,75.055
170652,0.6420,2020,0.13200,"['KEVVO', 'J Balvin']",0.856,189507,0.721,1,7HmnJHfs0BkFzX4x8j0hkl,0.004710,7,0.182,-4.928,1,Billetes Azules (with J Balvin),74,2020-10-16,0.1080,94.991


In [ ]:
df["artists"]

0         ['Sergei Rachmaninoff', 'James Levine', 'Berli...
1                                            ['Dennis Day']
2         ['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...
3                                          ['Frank Parker']
4                                            ['Phil Regan']
                                ...                        
170649                                         ['Ashnikko']
170650                                          ['MAMAMOO']
170651                                           ['Eminem']
170652                                ['KEVVO', 'J Balvin']
170653                                     ['Kyu Sakamoto']
Name: artists, Length: 170654, dtype: object

In [ ]:
df_w['artists']

0                         "Cats" 1981 Original London Cast
1                                "Cats" 1983 Broadway Cast
2              "Fiddler On The Roof” Motion Picture Chorus
3           "Fiddler On The Roof” Motion Picture Orchestra
4        "Joseph And The Amazing Technicolor Dreamcoat"...
                               ...                        
28675                                                  麥志誠
28676                                                  黃品源
28677                                                  黃國隆
28678                                                   黑豹
28679                                                  조정현
Name: artists, Length: 28680, dtype: object

In [ ]:
print("df:", df.shape)
print(df.head(1))
print("df_artist:", df_artist.shape)
print(df_artist.head(1))
print("df_genre:", df_genre.shape)
print(df_genre.head(1))
print("df_year:", df_year.shape)
print(df_year.head(1))
print("df_w:", df_w.shape)
print(df_w.head(1))

df_w.tail(1)

<img align="right" src="https://media.istockphoto.com/vectors/no-entry-sign-vector-id1128230184?k=6&m=1128230184&s=612x612&w=0&h=1Hfc4n5Gut2eqeC5fMcWGztghBXDfYCu_Y_9RddKpv8=" width=200>

# Look for file:
vectorized.csv

The following cell takes approximately 20 minutes to run.  As long as no changes have been made to the original dataframe, except for the addition of the 1 Japanese song, you can skip it and just do the following in a new cell:

```
df2 = pd.DataFrame("vecorized.csv")
```



In [ ]:
df['name_artist'] = df['name'] + df['artists']
df['name_artist']

0         Piano Concerto No. 3 in D Minor, Op. 30: III. ...
1                     Clancy Lowered the Boom['Dennis Day']
2         Gati Bali['KHP Kridhamardawa Karaton Ngayogyak...
3                                 Danny Boy['Frank Parker']
4                 When Irish Eyes Are Smiling['Phil Regan']
                                ...                        
170649              Halloweenie III: Seven Days['Ashnikko']
170650                                       AYA['MAMAMOO']
170651                                   Darkness['Eminem']
170652    Billetes Azules (with J Balvin)['KEVVO', 'J Ba...
170653    上を向いて歩こう Ue Wo Muite Arukou Sukiyaki['Kyu Saka...
Name: name_artist, Length: 170654, dtype: object

In [ ]:
# Making computer sense out of all these words
then = time.time() #Time before the operations start

print("Grab a cup of coffee, this is gonna take a while.")
x = [nlp(str(item)).vector for item in df['name_artist']]
now = time.time() #Time after it finished

z = (now-then)/60
print(f"It took: {z:0.1f} minutes to run this cell")


Grab a cup of coffee, this is gonna take a while.
It took: 18.7 minutes to run this cell


In [ ]:
df2 = pd.DataFrame(x)
print(df2.shape)
df2.head()

(170654, 300)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.134963,0.316618,0.030969,-0.126668,0.207609,0.047323,0.175922,-0.185210,-0.045755,0.980351,-0.267424,0.136610,0.117091,0.006647,-0.104592,0.107918,0.060590,0.898205,-0.102389,-0.011723,0.019577,0.158162,-0.027852,-0.017844,0.172549,-0.042341,-0.150844,0.095935,0.307635,0.042057,-0.065381,-0.015066,-0.071992,0.119081,0.093735,-0.057915,0.042746,0.074296,-0.035153,-0.053001,...,0.047125,0.051715,0.077381,0.169580,0.026577,-0.141432,0.158446,-0.077873,0.102083,-0.111606,-0.160838,-0.109636,-0.072945,-0.106083,-0.084104,0.039406,0.182049,-0.065946,-0.086841,0.184702,0.234030,-0.135740,0.153136,-0.142378,-0.067070,-0.031293,-0.056074,-0.139719,0.160565,-0.039220,0.158335,-0.044019,-0.039905,0.003677,0.192878,-0.005112,-0.147834,-0.015710,-0.159071,0.036169
1,-0.108814,0.134838,-0.101661,-0.036694,0.245011,-0.027559,-0.033172,0.019538,-0.090383,1.240873,-0.013691,-0.092558,0.252301,-0.069763,0.135833,0.045392,-0.118880,0.720914,-0.213291,-0.021886,-0.038294,-0.000173,0.070295,-0.097833,0.045104,-0.055305,-0.075099,0.026558,0.012220,0.005273,-0.059844,0.065384,0.086254,0.199598,0.148901,-0.046200,-0.027176,0.123559,0.053938,0.059419,...,0.014137,0.060272,-0.058733,0.093105,0.176352,-0.177732,0.085024,-0.036160,0.051750,-0.238019,0.164183,-0.073196,-0.052621,-0.131671,-0.001867,-0.099866,0.047601,0.164870,0.135869,0.087395,0.016152,-0.034726,-0.023559,-0.090944,0.022368,-0.009494,-0.025853,-0.113978,0.175366,0.071332,-0.079872,0.055137,0.047589,-0.025941,-0.054378,-0.151184,-0.250710,0.082874,-0.128079,-0.043889
2,-0.017790,0.064392,0.062554,-0.053999,0.145681,0.018219,0.057778,-0.129961,-0.081236,0.067503,-0.005168,0.102583,0.105809,0.010120,0.053924,0.122315,0.024359,0.190646,-0.003993,0.010581,-0.059604,0.156407,-0.001378,0.043727,0.083371,-0.019619,0.033576,0.051928,0.051366,0.016174,-0.039081,0.024725,-0.043968,0.022880,-0.022546,-0.033116,0.015773,0.123266,-0.186006,0.045708,...,-0.007288,-0.063512,0.077360,0.080286,0.122214,0.019136,0.053459,0.023366,0.071575,-0.170038,-0.075210,-0.026444,-0.038334,-0.002906,-0.045497,-0.080185,0.050662,0.018078,0.104310,0.091833,0.048856,-0.017465,0.060617,-0.082686,-0.025456,-0.066421,-0.068516,-0.060149,0.073615,-0.010551,0.055403,-0.063827,-0.002745,0.053326,0.011644,-0.057184,-0.074972,0.044888,-0.021594,-0.007073
3,-0.236132,0.248347,0.072515,-0.128750,0.203454,-0.099518,0.255106,-0.157495,-0.175258,0.458407,-0.141234,-0.177089,0.188786,-0.103601,-0.044561,0.212441,0.015143,0.541109,-0.000479,0.023511,-0.008712,0.122573,-0.022989,-0.003337,0.228202,-0.038580,-0.183416,0.106811,0.127582,-0.086554,-0.085383,0.209539,-0.104763,0.193766,0.028810,-0.071753,0.125912,0.186356,-0.155144,0.062508,...,-0.052276,-0.008787,0.033477,0.139581,0.101102,-0.292639,0.032892,0.095980,-0.031762,0.003632,-0.303251,0.157300,-0.037965,0.074208,0.017986,-0.119952,0.136986,-0.185094,-0.119900,0.135294,0.056500,0.020642,0.032233,0.190812,0.153796,-0.110744,0.046893,0.046055,0.153556,-0.154948,0.236719,-0.152676,0.049250,0.078352,0.167702,-0.153425,-0.285950,-0.026463,-0.206423,0.066826
4,-0.061122,0.098866,-0.086285,-0.080902,0.266051,-0.207497,0.091587,-0.035489,-0.119355,1.280177,-0.163280,0.079346,0.114230,-0.180726,0.012948,-0.093354,-0.054337,0.613637,-0.187848,-0.128394,0.044245,-0.015061,-0.084207,-0.141275,0.087337,-0.111003,-0.179364,0.043967,0.029227,-0.086706,-0.139910,0.000637,0.035794,0.113023,0.088803,-0.000658,-0.035758,-0.000092,-0.154284,0.049041,...,-0.093316,0.128707,-0.007040,0.190008,0.153057,-0.083155,0.055162,0.046254,0.078888,0.040803,-0.241977,0.081810,-0.168578,0.046698,0.037688,0.006221,0.040812,0.033844,0.067412,0.022902,0.137035,0.180544,-0.121237,0.056533,0.092020,-0.047862,0.035471,-0.015371,0.167016,-0.06724

In [ ]:
df2.to_csv("vectorized.csv")

# Start Playing Here!

In [ ]:
#@title Get the Model Right  { run: "auto" }
#@markdown ### Algorithm:
#@markdown ## Auto will choose for you
algorithm = "kd_tree" #@param ["ball_tree", "kd_tree", "brute", "auto"]
#@markdown ### Leaf Size default is 30
#@markdown ### This can affect the speed of the construction and query, as well as the memory requierd to store the tree.  The optimal value depends on the nature of the problem.
leaf_size = 30 #@param {type:"integer"}
#@markdown ### Metric  default = minkowski
metric = "minkowski" #@param {type:"string"}
#@markdown ### Metric Parameters
metric_params = "None" #@param {type:"string"}
#@markdown ### number of jobs
n_jobs =  1#@param {type:"integer"}
#@markdown # Nearest Neighbors
n_neighbors = 50 #@param {type:"integer"}

"""
Original Code From Yuan Jin
nn = NearestNeighbors(n_neighbors = 3, algorithm='ball_tree')
"""



nn = NearestNeighbors(algorithm=algorithm, leaf_size=leaf_size, 
                      n_neighbors=n_neighbors, n_jobs=n_jobs)




nn.fit(df2)



NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=1, n_neighbors=50, p=2, radius=1.0)

In [ ]:
#@title Default title text  { run: "auto" }
test = "harry bellafonte" #@param {type:"string"}


test_vect = nlp(test).vector
vect = test_vect.reshape(1, -1)
results = nn.kneighbors(vect)

s = [df['name'].iloc[x] for x in results[1][0]]
a = [df['artists'].iloc[x] for x in results[1][0]]
y = [df['release_date'].iloc[x] for x in results[1][0]]
     

# print(songs)
# print(results[1][0])
cn = ["Song", "Artist", "Year", "Row #"]
print(f"{cn[0]:<51}{cn[1]:<45}{cn[2]:<15}{cn[3]:^25}")
print("=" * 126)
for i in range(len(results[1][0])):

    print(f"{s[i][0:50]:<51}{a[i][0:44]:<45}{y[i]:^15}{results[1][0][i]:>16}")



Song                                               Artist                                       Year                     Row #          
Harry Potter - Prologue                            ['The London Fox Players']                     2010-11-29             168567
Harry Potter in 99 Seconds                         ['Jon Cozart']                                 2015-08-29             123822
Potter Waltz                                       ['Patrick Doyle']                              2005-11-09              54362
Harry in Winter                                    ['Patrick Doyle']                              2005-11-09              35678
Harry and Ginny                                    ['Alexandre Desplat']                          2010-11-23             138855
Harry & Hermione                                   ['Nicholas Hooper']                            2009-07-14              72895
Harry Hippie                                       ['Bobby Womack']                            